This notebook is inspired from @fchollet's notebook [tf.keras for Researchers: Crash Course](https://colab.research.google.com/drive/17u-pRZJnKN0gO5XZmq8n5A2bKGrfKEUg#scrollTo=uwttOFombD1F&forceEdit=true&sandboxMode=true). Great resource to learn about adding your custom layers, loss function, etc.

In [0]:
!pip install tensorflow==2.0.0

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

tf.__version__

'2.0.0'

# Create a new (custom) layer

In [0]:
class MyLayer(Layer):
    '''
    The __init__() method initializes the variables
    The call() method is where the logic or computation is defined
    '''
    
    def __init__(self, units=32, input_dim = 32):
        #Calling the constructor of the Layer class in tensorflow
        super(MyLayer, self).__init__()
        
        w_init = tf.random_normal_initializer()
        self.W = tf.Variable(initial_value=w_init(shape=(input_dim, units), dtype='float32'),
          trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,), dtype='float32'),
          trainable=True)
        
    def call(self, inputs):
        #Define computation here
        return tf.matmul(inputs, self.W) + self.b


In [5]:
#Instantiate our Layer
new_layer = MyLayer(4,2)
y = new_layer(tf.ones((2, 2)))
y

<tf.Tensor: id=28, shape=(2, 4), dtype=float32, numpy=
array([[-0.04563699, -0.02594603, -0.0306516 ,  0.02700859],
       [-0.04563699, -0.02594603, -0.0306516 ,  0.02700859]],
      dtype=float32)>

# add_weight method

In [0]:
class MyLayer(Layer):
    
    def __init__(self, units=32):
        super(MyLayer, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], self.units),
                               initializer='random_normal',
                               trainable=True, 
                                 name='{}_W'.format(self.name))
        self.b = self.add_weight(shape=(self.units,),
                               initializer='random_normal',
                               trainable=True,
                               name='{}_b'.format(self.name))
    def call(self, inputs):
        return tf.matmul(inputs, self.W) + self.b

# Nesting Layers


In [0]:
class MLP():

    def __init__(self):
        super(MLP, self).__init__()
        self.layer_1 = MyLayer(32) #MyLayer is a layer previously defined by us
        self.layer_2 = MyLayer(64)
        self.layer_3 = MyLayer(32)
        
    def call(self, inputs):
        x = self.layer_1(inputs)
        x = tf.nn.relu(x)
        x = self.layer_2(x)
        x = tf.nn.relu(x)
        
        return self.layer_3(x)
    
  

# Gradient Tape

Tensorflow provides GradientTape API for automatic differentiation. 

In [8]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x) #To observe the operations on x
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)
    
    dz_dx = t.gradient(z,x) #4x^3
    print(dz_dx.shape)
    
    for i in range(2):
        for j in range(2):
            print(dz_dx[i][j])

(2, 2)
tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)


# Adding a Loss layer

In [18]:
class LossClass(Layer):
  def __init__(self, rate):
    super(LossClass, self).__init__()
    self.rate = rate


  def call(self, inputs):

    # 'add_loss' is used to create a loss
    self.add_loss(self.rate * tf.reduce_sum(inputs))
    return inputs

#Using this loss in a MLP

class MLP2(Layer):
  """Stack of Linear layers with a sparsity regularization loss."""

  def __init__(self):
      super(MLP2, self).__init__()
      self.linear_1 = MyLayer(32)
      self.loss = LossClass(1e-2)
      self.linear_3 = MyLayer(10)

  def call(self, inputs):
      x = self.linear_1(inputs)
      x = tf.nn.relu(x)
      x = self.loss(x)
      return self.linear_3(x)
    
mlp = MLP2()
y = mlp(tf.ones((10, 10)))

print(mlp.losses)





[<tf.Tensor: id=134, shape=(), dtype=float32, numpy=0.24462295>]


# Creating a Dropout Layer

In [0]:
class Dropout(Layer):

  def __init__(self, rate=0.5):
    super(Dropout, self).__init__()
    self.rate = rate

  def call(self, inputs, training = None):
    if training:
      return tf.nn.dropout(inputs, rate = self.rate)
    return inputs

class MLPwithDropout(Layer):

  def __init__(self):

    super(MLPwithDropout, self).__init__()
    self.layer_1 = MyLayer(32)
    self.dropout = Dropout(0.5)
    self.layer_2 = MyLayer(10)

  def call(self, inputs, training = None):
    x = self.layer_1(inputs)
    x = tf.nn.relu(x)
    x = self.dropout(x, training = training)

    return self.layer_2(x)
  
obj = MLPwithDropout()
y_train = obj(tf.ones((2, 2)), training=True)
y_test = obj(tf.ones((2, 2)), training=False)